This is the main notebook to design the DnD Dungeon Master

Loading all relevant keys:

In [114]:
import os
from dotenv import load_dotenv

load_dotenv()

assert 'OPENAI_API_KEY' in os.environ, "You will need to set an OPENAI_API_KEY"
assert 'LANGCHAIN_API_KEY' in os.environ, "You will need to set an LANGCHAIN_API_KEY"

Importing all relevant libraries:

In [115]:
from langchain.agents import create_openai_functions_agent, create_react_agent, Tool
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.chains import LLMMathChain
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import AgentExecutor

import random

Specification from Canvas:

Project Requirements

World Memory: Implement persistent storage of state for each location. The AI must recognize when players revisit a location and present an updated description reflecting prior actions.

Map Generation and Navigation: The AI should create and maintain a coherent map of connected locations, ensuring logical placement and connections.

Dynamic Descriptions and Interactions: Generate unique descriptions and manage NPC or item states dynamically, adapting based on player actions.
Randomized Location Descriptions and Encounters: When players enter a new location, the AI generates a fresh, random description consistent with nearby areas and may spawn new NPCs or items. This allows each exploration to feel novel and immersive while maintaining a coherent world.

Simplified D&D Mechanics: Use a streamlined set of D&D-inspired mechanics (detailed below) to guide the AI’s decisions and narration for combat, exploration, and interactions.

DnD Simplified Mechanics:

NPC Interaction and Combat
NPCs have two attributes: Health Points (HP) and Attack Power.
Combat: When players engage in combat, each side rolls a virtual 1d6 (a six-sided die) and adds it to their Attack Power to determine damage for that turn.

Victory Conditions: The player wins if the NPC’s HP reaches zero, and loses if their own HP does.
Skill Checks and Random Outcomes
When players attempt a risky action (e.g., unlocking a door), the AI rolls a 1d10 (a ten-sided die) for a skill check.
For simple tasks, a roll of 3 or higher succeeds.
For challenging tasks, a roll of 6 or higher is needed.
Descriptions should vary based on success or failure, giving the interaction a creative touch.

Inventory and Simple Item Use
Players can pick up and use basic items like keys or healing potions.
Items: Each item has a single effect—e.g., a potion heals 10 HP, a key unlocks a specific door.
The AI remembers which items have been used or taken to manage inventory without complex tracking.

Experience Points and Leveling (Optional)
Players can gain Experience Points (XP) for completing significant actions or defeating NPCs.
When players reach 50 XP, they "level up," gaining a small HP increase.
This provides a sense of progression with minimal complexity.

In [ ]:
@tool
def rolld6(a: int) -> int:
    """ Roll a six-sided dice
    """
    rand = random.randint(1, 6)
    return rand

@tool
def rolld10(a: int) -> int:
    """ Roll a ten-sided dice
    """
    rand = random.randint(1, 10)
    return rand

@tool
def combat(player_health: int, player_attack: int, npc_health: int, npc_attack: int) -> {int, int}:
    """ Player enters combat with an NPC
    args:
        player_health: player's remaining health
        player_attack: player's attack power
        npc_health: npc's remaining health
        npc_attack: npc's attack power
    """
    while (player_health > 0 and npc_health > 0):
        player_trueap = player_attack + rolld6({})
        npc_trueap = npc_attack + rolld6({})
        player_health -= npc_trueap
        npc_health -= player_trueap

    return player_health, npc_health

tools = [rolld6, rolld10]

State Persistence

The AI must track and remember states for each location and NPC to maintain continuity. This includes:

Location State: Track if a door is unlocked, a chest is opened, or an item has been taken.

NPC Status: Remember if NPCs have been defeated, moved, or interacted with.

Inventory: Track items players possess and mark items as “used” once they’re applied (e.g., a used key).

Player Stats: Keep player attributes like HP, XP, and inventory status persistent across moves.


Map Representation

The map is a logical grid or node-based structure, where each “node” represents a unique location.

Location Connections: Each node includes connections to adjacent locations (e.g., north, south, east, west).

Unique Descriptions and Random Encounters: Each time players enter a new node, the AI generates a fresh description that is contextually consistent with nearby areas. The AI may also spawn random NPCs or items, creating an immersive experience while ensuring coherence.

NPC and Item Placement: Each node may contain NPCs or items, which the AI can add or remove based on player actions.

GRADUATE STUDENT:
Graduate students are required to implement additional features to expand the AI’s storytelling capabilities:

Text-to-Speech (TTS): Add a TTS system to narrate the AI-generated descriptions and interactions, enhancing the immersive experience.

Image Generation: Integrate a simple image generation model to create visual representations of locations or key events, adding a visual storytelling element to the text-based game.

In [117]:
llm = ChatOpenAI(temperature = 0, model='gpt-3.5-turbo-1106')

Prompt Template

In [118]:
prompt = hub.pull("hwchase17/react")
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

Use AI Agent

In [119]:
# Create an OpenAI Functions agent
agent = create_react_agent(llm, tools, prompt=prompt)
# Create an agent executor
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent,tools=tools,verbose=True, handle_parsing_errors=True)

In [120]:
# Test the agent
question = "Run the test and print out the result"
response = agent_executor.invoke({"input": question})
print(f"Question: {question}")
print(f"Answer: {response}")



> Entering new AgentExecutor chain...
I need to use the testTool to get the result
Action: testTool
Action Input: 5Hello WorldThe testTool is working properly
Final Answer: Hello World

> Finished chain.
Question: Run the test and print out the result
Answer: {'input': 'Run the test and print out the result', 'output': 'Hello World'}


In [121]:
# Test the agent
question = "Player rolls a d6, and NPC rolls a d6. Who wins?"
response = agent_executor.invoke({"input": question})
print(f"Question: {question}")
print(f"Answer: {response}")



> Entering new AgentExecutor chain...
We need to simulate the rolls for both the player and the NPC to determine the winner.
Action: rolld6
Action Input: 13The player rolled a 3.
Action: rolld6
Action Input: 16The NPC rolled a 6.
Final Answer: The NPC wins.

> Finished chain.
Question: Player rolls a d6, and NPC rolls a d6. Who wins?
Answer: {'input': 'Player rolls a d6, and NPC rolls a d6. Who wins?', 'output': 'The NPC wins.'}


In [122]:
scenario = "Player has 100 health and 20 attack power. NPC has 200 health and 10 attack power. "
question = "Player fights NPC until one's health reaches zero. Who wins, and how much health do they have left?"
response = agent_executor.invoke({"input": scenario+question})
print(f"Response: {response}")



> Entering new AgentExecutor chain...
I need to simulate the battle between the player and the NPC to see who wins and how much health they have left.
Action: testTool
Action Input: "Player has 100 health and 20 attack power. NPC has 200 health and 10 attack power."

ValidationError: 1 validation error for testTool
a
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='Player has 100 health an...th and 10 attack power.', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_parsing